# Análisis Cluster: Países del Mundo

En  este notebook vamos a proceder al cálculo de clusters para el dataset con información de los distintos países. Nos apoyaremos en el cálculo de Componentes Principales hechos en el notebook anterior para generar dichos clusters, por lo que la primera parte de este notebook será idéntica al anterior.

<div>
<img src="./media/mapamundi.jpg" width="500"/>
</div>

Comenzamos importando los módulos:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Comenzamos cargando los datos en un dataframe:

In [ ]:
paises_data = pd.read_csv('./data/country_data.csv', index_col = 0)

Como siempre nos aseguramos de que los datos se hayan cargado correctamente:

In [ ]:
paises_data.head()

In [ ]:
paises_data.shape

De nuevo observamos que existen muchas diferencias entre las escalas de las variables por lo que pasamos como siempre a escalar los datos:

In [ ]:
from sklearn.preprocessing import scale
X_paises = pd.DataFrame(scale(paises_data), index=paises_data.index, columns=paises_data.columns)

Con esto hemos generado nuestros datos escalados:

In [ ]:
X_paises.head()

Una vez hecho esto podemos comenzar calculando nuestros vectores de carga:

Tenemos en total 9 componentes pricipales. Esto era de esperar pues si recordamos nuestro dataframe tiene 167 observaciones y 9 variables luego tendremos un total de min (167-1,9) componentes principales.

Podemos construir una tabla en la que vemos como se proyectan los países sobre las nuevas coordenadas:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
df_plot = pd.DataFrame(pca.fit_transform(X_paises), columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9'], index=X_paises.index)
df_plot.head()

Para evaluar la calidad de este modelo podemos comprobar cuánta varianza se explica en cada componente:

In [ ]:
pca.explained_variance_

El número fuera de contexto puede resultar confuso por lo que observamos en su lugar el ratio de varianza explicada:

In [ ]:
pca.explained_variance_ratio_

Puede resultar interesante también observar la suma acumulada de estos ratios:

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

Visualizamos este último dato mediante un gráfico:

Personalmente a la hora de elegir el número de componentes siempre me ha parecido más intuitivo el gráfico de varianza acumulada:

In [ ]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3,4,5,6,7,8,9], np.cumsum(pca.explained_variance_ratio_), '-s')
plt.ylabel('Proporción de varianza explicada acumulada')
plt.xlabel('Componentes Principales')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3,4,5,6,7,8,9]);

En el notebook previo vimos que con esta información llegaba el momento de calcular el número de componentes principales adecuado. Observamos que con las primeras 5 componentes, estamos expresando casi el total de la varianza (un 95%) por lo que emplearemos cinco componentes principales:

In [ ]:
df_cluster = df_plot[['PC1', 'PC2', 'PC3', 'PC4', 'PC5']]
df_cluster.head()

Una vez ya hemos extraído las componentes principales y hemos decidido quedarnos con cinco procedemos a la división en clusters. Comenzaremos con el clustering no jerárquizado.

## Cluster no jerárquico

Existen varios métodos de clustering no jerarquizado pero el más popular sin lugar a dudas es el método K-means  que ya hemos estudiado en la parte teórica de esta sección. A continuación procedemos a su implementación.

Como sabemos en los métodos no jérarquicos debemos decidir a priori el número de clusters, para tomar dichas decisión podemos basarnos en los gráficos de sedimentación:

In [ ]:
from sklearn.cluster import KMeans
ssd = []
for num_clusters in list(range(1,10)):
    model_clus = KMeans(n_clusters = num_clusters, max_iter=50)
    model_clus.fit(df_cluster)
    ssd.append(model_clus.inertia_)

plt.plot(ssd)
plt.show()

La curva nos indica que pueden ser interesantes 3 clusters o 5. Elegiré 3 clusters pero puede ser interesante como ejercicio estudiar que ocurre si tomamos cinco:

In [ ]:
numero_clusters = 3

Instanciamos el modelo fijando el número máximo de iteraciones y el número máximo de clusters así como una semilla aleatoria cuyo propósito ya vimos en la sección previa:

In [ ]:
cluster_model = KMeans(n_clusters = numero_clusters, max_iter=50,random_state = 50)

Una vez instanciado el modelo lo ajustamos sobre nuestros datos:

In [ ]:
cluster_model.fit(df_cluster)

Podemos observar las etiquetas generadas para cada observación:

In [ ]:
pd.Series(cluster_model.labels_)

Unimos las etiquetas con los datos de las componentes principales con las que fueron calculados:

In [ ]:
df_no_index = df_cluster

In [ ]:
df_no_index.reset_index(level=0, inplace=True)

In [ ]:
clustered_components = pd.concat([df_no_index, pd.Series(cluster_model.labels_)], axis=1)

In [ ]:
clustered_components.rename(columns={0:'Cluster'})

Con fines de interpretación unimos dicha etiqueta de cluster con los datos iniciales para poder estudiar las características de los clusters más cómodamente:

In [ ]:
merging_data = pd.merge(paises_data, clustered_components.set_index('country'), left_index=True,right_index=True).rename(columns={0:'Cluster'})

In [ ]:
merging_data.head()

In [ ]:
interpretable_data = merging_data[['child_mort', 'exports', 'health', 'imports', 'income', 'inflation', 'life_expec', 'total_fer', 'gdpp', 'Cluster']]

Estos son nuestros datos interpretables:

In [ ]:
interpretable_data.head()

Podemos ir inspeccionando cluster por cluster:

In [ ]:
interpretable_data[interpretable_data.Cluster==0].head(20)

En este primer cluster observamos que nos encontramos con países desarrollados en los que observamos por ejemplo altos ingresos y una tasa de mortalidad infantil relativamente baja. Podemos agruparlos en un dataframe:

In [ ]:
paises_desarrollados = interpretable_data[interpretable_data.Cluster==0]

In [ ]:
interpretable_data[interpretable_data.Cluster==1].head(20)

En este segundo cluster observamos los países menos desarrollados con una tasa de mortalidad infantil muy elevada y un PIB muy bajo. Podemos agruparlos en un dataframe. que posteriormente exploraremos:

In [ ]:
paises_no_desarrollados = interpretable_data[interpretable_data.Cluster==1]

In [ ]:
interpretable_data[interpretable_data.Cluster==2].head(20)

Este último cluster agrupa los países intermedios, algunos de ellos se suelen definir como países en vía de desarrollo por la literatura.

Observamos que los clusters no son perfectos,  por ejemplo, países como Argentina o Chile quizás irían mejor en nuestro primer cluster que en este. Esta es la parte negativa de la flexibilidad  de los métodos no supervisados.

In [ ]:
paises_en_desarrollo = interpretable_data[interpretable_data.Cluster==2]

## Explorando los clusters

Para comprender mejor los clusters podemos calcular la media agregada de cada variable por cluster:

In [ ]:
Cluster_GDPP=pd.DataFrame(interpretable_data.groupby(["Cluster"]).gdpp.mean())
Cluster_child_mort=pd.DataFrame(interpretable_data.groupby(["Cluster"]).child_mort.mean())
Cluster_exports=pd.DataFrame(interpretable_data.groupby(["Cluster"]).exports.mean())
Cluster_income=pd.DataFrame(interpretable_data.groupby(["Cluster"]).income.mean())
Cluster_health=pd.DataFrame(interpretable_data.groupby(["Cluster"]).health.mean())
Cluster_imports=pd.DataFrame(interpretable_data.groupby(["Cluster"]).imports.mean())
Cluster_inflation=pd.DataFrame(interpretable_data.groupby(["Cluster"]).inflation.mean())
Cluster_life_expec=pd.DataFrame(interpretable_data.groupby(["Cluster"]).life_expec.mean())
Cluster_total_fer=pd.DataFrame(interpretable_data.groupby(["Cluster"]).total_fer.mean())

In [ ]:
aggregated_mean = pd.concat([Cluster_GDPP,Cluster_child_mort,Cluster_income,Cluster_exports,Cluster_health,
                Cluster_imports,Cluster_inflation,Cluster_life_expec,Cluster_total_fer], axis=1)

In [ ]:
aggregated_mean

Observamos como el primer cluster (cluster 0) formado por los llamados países desarrollados aglutina los mayores PIB e ingresos junto con la mayor esperanza de vida y una mortalidad infantil relativamente baja. También tiene una tasa de fertilidad muy baja. 

El cluster 1 dispara la mortalidad infantil y también la tasa de fertilidad (ya vimos que estaban correlacionadas). También sube la inflación mientras que se desploman el PIB y la esperanza de vida.

En el último cluster (cluster 2) observamos que la esperanza de vida es bastante alta y en general resulta un punto intermedio entre los otros dos clusters.

Podemos construir gráficos de barras para observar cómo se distribuyen algunas varibles según el cluster:

In [ ]:
fig = plt.figure(figsize = (10,6))
aggregated_mean.rename(index={0: 'Países desarrollados'},inplace=True)
aggregated_mean.rename(index={2: 'Países en vía de desarrollo'},inplace=True)
aggregated_mean.rename(index={1: 'Países subdesarrollados'},inplace=True)

s=sns.barplot(x=aggregated_mean.index,y='gdpp',data=aggregated_mean)
plt.xlabel('Clusters de países', fontsize=10)
plt.ylabel('PIB per Capita', fontsize=10)
plt.title('Clusters en base al PIB')
plt.show()


In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x=aggregated_mean.index,y='child_mort',data=aggregated_mean)
plt.xlabel('Clusters de países', fontsize=10)
plt.ylabel('Mortalidad infantil', fontsize=10)
plt.title('Clusters en base a la mortalidad infantil')
plt.show()

Con esta misma idea se pueden construir gráficos de cajas:

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='Cluster',y='income',data=interpretable_data)
plt.xlabel('Clusters de países', fontsize=10)
plt.ylabel('Ingresos', fontsize=10)
plt.title('Ingresos per capita de cada cluster')
plt.show()

También podemos observar como se distribuye una variable dentro de las observaciones agrupadas en un mimsmo cluster:

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=paises_desarrollados.index,y='child_mort',data=paises_desarrollados)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('País', fontsize=10)
plt.ylabel('Mortalidad infantil', fontsize=10)
plt.title('Mortalidad infantil en países desarrollados ')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=paises_no_desarrollados.index,y='child_mort',data=paises_no_desarrollados)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('País', fontsize=10)
plt.ylabel('Mortalidad infantil', fontsize=10)
plt.title('Mortalidad infantil en países subdesarrollados  ')
plt.show()

Estos gráficos podrían constituir un buen informe por ejemplo para una institución que esté buscando en qué países puede ser más interesante implantar programas de ayuda para la población y en qué tipo de problemas podrían centrarse estos programas.

A continuación construiremos un clustering jerarquizado para ver cuál arroja mejores resultados.

## Clustering jerárquico

Retomamos los datos de las componentes principales:

In [ ]:
df_cluster

Comenzamos construyendo un primer dendograma en el que usamos como linkage el método simple. Como distancia especificamos la euclíea:

In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram


mergings_average=linkage(df_cluster.set_index('country'),method='single',metric='euclidean')
fig = plt.figure(figsize = (34,15))
dendrogram(mergings_average)
plt.show()

En un principio no parece muy buena idea, va cogiendo siempre clusters con países sueltos. Probamos un tipo distinto de vínculo, por ejemplo el completo:

In [ ]:
mergings_complete=linkage(df_cluster.set_index('country'),method='complete',metric='euclidean')
fig = plt.figure(figsize = (34,15))
dendrogram(mergings_complete)
plt.show()

Este dendograma tiene mejor aspecto pese a que siga dejando a un país aislado en un solo cluster. Procedemos a continuación a cortar el árbol para generar los árboles. Parece interesante cortar en cuatro clusters:

In [ ]:
from scipy.cluster.hierarchy import cut_tree
etiquetas_cluster=cut_tree(mergings_complete,n_clusters=4).reshape(-1,)

Observamos las etiquetas generadas por este método:

In [ ]:
etiquetas_cluster

Asociamos esta etiqueta a los datos para estudiar los clusters como hicimos previamente:

In [ ]:
df_cluster['ClusterID'] = etiquetas_cluster

In [ ]:
merging_data = pd.merge(paises_data, df_cluster.set_index('country'), left_index=True,right_index=True).rename(columns={0:'Cluster'})

In [ ]:
merging_data.head(10)

Nos quedamos solo con las variables que nos interesan  descartando las componentes principales por su nula interpretabilidad:

In [ ]:
data_jerarquico = merging_data[['child_mort', 'exports', 'health', 'imports', 'income', 'inflation', 'life_expec', 'total_fer', 'gdpp', 'ClusterID']]

In [ ]:
data_jerarquico

Una vez más estudiamos los clusters

In [ ]:
data_jerarquico[data_jerarquico.ClusterID==0].head(20)

In [ ]:
data_jerarquico[data_jerarquico.ClusterID==1]

In [ ]:
data_jerarquico[data_jerarquico.ClusterID==2]

In [ ]:
data_jerarquico[data_jerarquico.ClusterID==3]

Observamos que el primer cluster agrupa a los países generales, en el segundo tenemos a los países desarrollados, en el tercero a países bastante ricos y en el cuarto cluster nos ha quedado el país aislado que ha resultado ser Nigeria.

Agregamos los datos para sacar alguna conclusión más:

In [ ]:
Cluster_GDPP=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).gdpp.mean())
Cluster_child_mort=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).child_mort.mean())
Cluster_exports=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).exports.mean())
Cluster_income=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).income.mean())
Cluster_health=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).health.mean())
Cluster_imports=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).imports.mean())
Cluster_inflation=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).inflation.mean())
Cluster_life_expec=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).life_expec.mean())
Cluster_total_fer=pd.DataFrame(data_jerarquico.groupby(["ClusterID"]).total_fer.mean())

In [ ]:
media_agregada_jerar = pd.concat([Cluster_GDPP,Cluster_child_mort,Cluster_income,Cluster_exports,Cluster_health,
                Cluster_imports,Cluster_inflation,Cluster_life_expec,Cluster_total_fer], axis=1)

In [ ]:
media_agregada_jerar

Observamos que el cluster 2 podrían ser los países con más calidad de vida: tienen mayor esperanza de vida, menor mortalidad infantil e ingresos muy altos.  El cluster 1 agrupa países con una gran inversión en salud, una alta esperanza de vida y una mortalidad infantil también baja. Así podriamos agrupar los datos en los siguientes dataframes:

In [ ]:
paises_general = data_jerarquico[data_jerarquico.ClusterID==0]

In [ ]:
paises_desarrol_jerar = data_jerarquico[data_jerarquico.ClusterID==1]

In [ ]:
paises_ricos_jerar = data_jerarquico[data_jerarquico.ClusterID==2]

In [ ]:
nigeria_jerar = [data_jerarquico.ClusterID==3]

De nuevo podemos construir algunos gráficos para visualizar nuestros resultados

In [ ]:
fig = plt.figure(figsize = (10,6))
media_agregada_jerar.rename(index={0: 'Grupo general'},inplace=True)
media_agregada_jerar.rename(index={1: 'Grupo desarrollado'},inplace=True)
media_agregada_jerar.rename(index={2: 'Grupo rico'},inplace=True)
media_agregada_jerar.rename(index={3: 'Grupo Nigeria'},inplace=True)
s=sns.barplot(x=media_agregada_jerar.index,y='gdpp',data=media_agregada_jerar)
plt.xlabel('Clusters de países', fontsize=10)
plt.ylabel('PIB per Capita', fontsize=10)
plt.title('Clusters en base a su PIB')
plt.show()



Que Nigeria aparezca el más bajo no significa que sea el país con menor PIB recordemos que en el grupo general es en el que más países hay y lo reflejado es la media de PIB.

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x=media_agregada_jerar.index,y='child_mort',data=media_agregada_jerar)
plt.xlabel('Clusters de países', fontsize=10)
plt.ylabel('Mortalidad infantil', fontsize=10)
plt.title('Clusters de países en base a la mortalidad infantil')
plt.show()

Podemos de nuevo construir gráficos de cajas:

In [ ]:
fig = plt.figure(figsize = (12,8))
sns.boxplot(x='ClusterID',y='income',data=data_jerarquico)
plt.xlabel('Cluster de países', fontsize=10)
plt.ylabel('Ingresos por persona', fontsize=10)
plt.title('Ingresos por persona en cada cluster')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=paises_ricos_jerar.index,y='gdpp',data=paises_ricos_jerar)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('País', fontsize=10)
plt.ylabel('PIB', fontsize=10)
plt.title('PIB en países ricos')
plt.show()

In [ ]:
fig = plt.figure(figsize = (18,6))
s=sns.barplot(x=paises_desarrol_jerar.index,y='gdpp',data=paises_desarrol_jerar)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.xlabel('País', fontsize=10)
plt.ylabel('PIB', fontsize=10)
plt.title('PIB en países desarrollados')
plt.show()

## Conclusiones

Ahora que hemos estudiado ambos métodos podemos concluir que en este problema ofrece mejores resultados el método no jerárquico pues produce una división muy razonable tanto en tamaños (los clusters son similares) como en resultados a la hora de realizar un análisis más profundo.

El clustering jerárquico nos devuelve cuatro clusters muy irregulares (uno formado solo por Nigeria, otro formado por tres países...) Como estudiamos en la parte teórica el desarrollo jerárquico pese a su interpretabilidad no se ajusta adecuadamente a un gran número de problemas.